In [1]:
from datetime import *
from statistics import mean 
from itertools import product
import sys 
import os
path = os.path.dirname(os.getcwd())
sys.path.append(path)
from lib import *

# Baseline

In [2]:
data, vars_ms, userMax, energyDemand, q = getConstantsFromBPMN('../flightBooking.json')
# User Data
df = pd.read_csv(r'../data/projectcount_wikiDE_2015.csv')
clickData_hourly = df["De"].tolist()
clickData_hourly = clickData_hourly[24:] + clickData_hourly[0:24]

## Low Power

In [3]:
ed_hourly_lowPower = []
prodQ = 1.0
for i in range(8760):
    ed = 0
    s = clickData_hourly[i]
    for j in range(len(q)):
        if userMax[j][0] > 0:
            ed += energyDemand[j][0] * math.ceil(s / userMax[j][0])
        s = s * q[j][0]

    ed_hourly_lowPower.append(ed)

prodQ = 1.0
for j in range(len(q)):
    if q[j][0] > 0.1:
        prodQ *= q[j][0]
print("Q: " + str(prodQ))

Q: 0.1670625


# Normal
Here the applications architecture was picked such that it's energy consumption lies in the middle of the spectrum. The architecture is not optimized for energy consumption, but also not for performance.

In [4]:
lowPower_ed = sum(energyDemand[i][0] for i in range(len(energyDemand)))
highPower_ed = sum(energyDemand[i][-1] for i in range(len(energyDemand)))

median_ed = (highPower_ed - lowPower_ed)/2


indices = []
for ms in data['components']:
    indices.append([0] * len(data['components'][ms]))

# Calculating all possible configurations
vars_gb = [[]]
for i in indices:
    vars_gb *= len(i)
ed_combo = list(product(*energyDemand))
userMax_combo = list(product(*userMax))
q_combo = list(product(*q))

for i in range(len(vars_gb)):
    vars_gb[i].append([ed_combo[i], userMax_combo[i], q_combo[i]])

In [5]:
# Finding the configuration with the closest energy demand to the median energy demand
temp = highPower_ed
for i in range(len(vars_gb)):
    if abs(sum(vars_gb[0][i][0])-median_ed) < temp:
        temp = abs(sum(vars_gb[0][i][0])-median_ed)
        index = i
normal_conf = vars_gb[0][index]

In [6]:
ed_hourly_normal = []
for i in range(8760):
    ed = 0
    s = clickData_hourly[i]
    for j in range(len(q)):
        if normal_conf[1][j] > 0:
            ed += normal_conf[0][j] * math.ceil(s / userMax[j][0])
        s = s * normal_conf[2][j]
    ed_hourly_normal.append(ed)

prodQ = 1.0
for j in range(len(q)):
        prodQ *= normal_conf[2][j]
print("Q: " + str(prodQ))

Q: 0.30071250000000005


## High Performance

In [7]:
ed_hourly_highPower = []
for i in range(8760):
    ed = 0
    s = clickData_hourly[i]
    for j in range(len(q)):
        if userMax[j][-1] > 0:
            ed += energyDemand[j][-1] * math.ceil(s / userMax[j][-1])
        s = s * q[j][-1]
    ed_hourly_highPower.append(ed)
prodQ = 1.0
for j in range(len(q)):
    if q[j][-1] > 0.1:
        prodQ *= q[j][-1]
print("Q: " + str(prodQ))

Q: 0.8464499999999999


In [8]:
# new 03.07.23
df = pd.read_csv(r'../data/DE_2021.csv')
ci_data_2021_hourly = df['carbon_intensity_avg']

ed_daily_highPower = []
for i in range(365):
    temp = 0
    for j in range(24):
        temp += ed_hourly_highPower[i*24+j]*ci_data_2021_hourly[i*24+j]
    ed_daily_highPower.append(temp)

In [9]:
ed_hourly_lowPower = np.array(ed_hourly_lowPower)
#ed_hourly_highPower[1000] = 6277312.0
#ed_hourly_highPower[1001] = 6277312.0
ed_hourly_highPower = np.array(ed_hourly_highPower)

data = pd.DataFrame({'lp': ed_hourly_lowPower, 'hp': ed_hourly_highPower}, columns=['lp', 'hp'])

# Write Data to CSV

In [10]:
f = open('../results/baseline.csv', 'w')
writer = csv.writer(f)
row = ["lp","np","hp"]
writer.writerow(row)
for lp,np,hp in zip(ed_hourly_lowPower,ed_hourly_normal,ed_hourly_highPower):
    row = []
    row.append(lp)
    row.append(np)
    row.append(hp)
    writer.writerow(row)
f.close()